In [ ]:
from telethon import TelegramClient
import telethon
from telethon import functions, utils
from telethon.tl.types import Message, Channel, InputMessagesFilterEmpty, InputPeerEmpty, PeerChannel
from telethon.tl.custom.dialog import Dialog
from telethon.tl.types.messages import DialogFilters
from telethon.tl.custom.file import File
from telethon.tl.types.messages import MessagesSlice
from tqdm import tqdm
from dotenv import load_dotenv
from typing import List, Dict, Any
import os
import asyncio
import json
from datetime import date, datetime
from telethon.errors import FloodWaitError
from sqlalchemy.engine.base import Engine
from sqlmodel import create_engine
from models.telegram import MediaItem, ChannelModel
from lib.tl_client import process_with_backoff, get_context

from pprint import pp

os.chdir(os.path.join(os.getcwd(), ".."))

load_dotenv()

ctx = await get_context()


## Experiments and scratch

In [ ]:

# Manually connect
client = TelegramClient(session_file, api_id, api_hash)
print(api_id, api_hash)
await client.connect()
print(client.is_connected())
is_authorized = await client.is_user_authorized()
print(is_authorized)
me = await client.get_me()
print(me.stringify())
engine = create_engine(f"sqlite:///{DB_PATH}")


In [ ]:
# Browse all chats

dialogs: List[Dialog] = await ctx.tclient.get_dialogs()

dialog_by_id = {}
for d in dialogs:
    real_id, peer_type = utils.resolve_id(d.id)
    print(real_id, peer_type)
    dialog_by_id[real_id] = d

# client.get_channels(dialog_by_id.keys())

In [ ]:
# Find recent messages in a chat
filtered = filter(lambda dialog: dialog.name.find('Abrahams') > -1, dialogs)
f: Dialog = next(filtered)
last_messages = await ctx.tclient.get_messages(f, limit=20)

[print(m) for m in last_messages]

In [ ]:
# Delete most recent message
# await client.delete_messages(f, last_messages[0], revoke=True)

In [ ]:
# Find folders
chatFolders: DialogFilters = await ctx.tclient(functions.messages.GetDialogFiltersRequest())
# print(chatFolders.stringify())

In [ ]:
# Find chats in media folder
mediaFolder = None
for folder in chatFolders.filters:
    if not hasattr(folder, 'title'):
        continue
    if folder.title == 'MediaView':
        mediaFolder = folder
        break

# print(mediaFolder.stringify())


# target_channels = [dialog_by_id[peer.channel_id] for peer in mediaFolder.include_peers]
# target_channels = [client.get_entity(peer) for peer in mediaFolder.include_peers]
# await client.get_entity(peer)

target_channels = await asyncio.gather(*[client.get_entity(peer) for peer in mediaFolder.include_peers])
for channel in target_channels:
    print(channel.stringify())

In [ ]:
# Test messages in the first two channels
"""
for channel_id in target_channels[0:2]:
    async for message in ctx.tclient.iter_messages(channel_id, limit=5):
        print(message.stringify())
"""

# Grab most recent post @ self to test twitter embed
test_message = None
async for message in ctx.tclient.iter_messages('me', limit=1):
    test_message = message

print(test_message.stringify())

# test_message.media = MessageMediaWebPage
print(test_message.document)

In [ ]:
# Grab posts in channel

async def collect_posts(target_channels, limit=5):
    post_collection = []
    for channel in target_channels:
        async for message in ctx.tclient.iter_messages(channel, limit=limit):
            post_collection.append(message)
    return post_collection
posts = await collect_posts(target_channels, limit=5)

# Examine individual messages

In [ ]:
channel_title = 'GIFs and Vids'

async def find_channel() -> telethon.types.Dialog | None:
    async for dialog in ctx.tclient.iter_dialogs():
        d: telethon.types.Dialog = dialog
        if d.title.find(channel_title) > -1:
            print(d)
            return d
    print("Channel not found.")
    return None

target_channel = await find_channel()


In [ ]:
forwardMessage = None
async for message in ctx.tclient.iter_messages(target_channel, 500):
    if message.forward:
        forwardMessage = message
        print(message.stringify())
        break

print(await forwardMessage.forward.get_input_chat())
print(forwardMessage.peer_id.channel_id)
print(forwardMessage.forward.original_fwd.from_id)


In [ ]:

messages = await ctx.tclient.get_messages(target_channel, 2)

In [ ]:


message = messages[0]
print(message.stringify())


In [ ]:

message
print(message.media.webpage.has_large_media) # true
web_document = message.media.webpage.document
print(web_document.mime_type == 'video/mp4') # true
dir(web_document)
# dir(message)
print(message.file.mime_type)
web_document.id
# print(message.file)
# print(dir(message.file))
print(message.file.media.id)
#print(message.video.stringify()) # 5824722488059233779

In [ ]:
dl = await ctx.tclient.download_media(web_document, MEDIA_PATH)


In [ ]:
dl

# Search

In [ ]:
twitter_users = json.load(open('./data/twitter_users.json'))
twitter_users

In [ ]:


def makeSearchRequest(query: str, limit: int = 50):
    return telethon.functions.messages.SearchGlobalRequest(
        q=query,
        filter=InputMessagesFilterEmpty(),
        min_date=-1,
        max_date=-1,
        offset_rate=0,
        offset_peer=InputPeerEmpty(),
        offset_id=-1,
        limit=limit,
    )

channel_ids = set()


In [ ]:
print(results.stringify())

In [ ]:

for twitter_user in tqdm(twitter_users):
    searchRequest = makeSearchRequest(twitter_user, 50)
    results: MessagesSlice = await client(searchRequest)
    for message in results.messages:
        if isinstance(message.peer_id, PeerChannel):
            channel_ids.add(message.peer_id.channel_id)


In [ ]:
channel_ids = sorted(list(channel_ids))
# with open('./data/searched_channels.json', 'w') as f:
    # json.dump(list(channel_ids), f, indent=2)

In [ ]:

async def id_to_channel(client, channel_id):
    channel = await client.get_entity(channel_id)
    return channel


channels: List[Channel] = await asyncio.gather(*[id_to_channel(client, channel_id) for channel_id in channel_ids])


In [ ]:
print(sorted([channel.username for channel in channels if channel.username is not None]))